# SVM model for Fall Detection

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import precision_score, accuracy_score
import warnings
import os
import random
import time

random.seed(42)
warnings.filterwarnings('ignore')

Loading data

In [2]:
train_df = pd.read_csv("C:/Users/dawou/OneDrive/Bureau/ML/ChuteDetc/Train.csv")
test_df = pd.read_csv('C:/Users/dawou/OneDrive/Bureau/ML/ChuteDetc/Test.csv')

# Dropping the unnecessary 'Unnamed: 0' column
train_df.drop(['Unnamed: 0'], axis=1, inplace=True)
test_df.drop(['Unnamed: 0'], axis=1, inplace=True)

# Splitting the data into features and target
X_train = train_df.drop(['fall', 'label'], axis=1)
y_train = train_df['fall']
X_test = test_df.drop(['fall', 'label'], axis=1)
y_test = test_df['fall']

Defining the grid of hyperparameters for SVM

In [14]:
random_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    #'kernel': ['rbf', 'poly', 'sigmoid']
    'kernel': ['poly']
}

Creating the SVM classifier using sklearn

In [15]:
from sklearn.svm import SVC
svm = SVC()

Performing Randomized Search

In [16]:
svm_random = RandomizedSearchCV(estimator=svm,
                                param_distributions=random_grid,
                                n_iter=100,
                                cv=9,
                                verbose=2,
                                random_state=42,
                                n_jobs=-1)
start_time = time.time()
svm_random.fit(X_train, y_train)
training_time = time.time() - start_time

Fitting 5 folds for each of 48 candidates, totalling 240 fits


Best parameters from the search

In [17]:
best_params = svm_random.best_params_
print(best_params)

{'kernel': 'poly', 'gamma': 0.01, 'C': 1}


Evaluating the mode

In [19]:
# Evaluating the model
def evaluate_svm(model, test_features, test_labels):
    start_time = time.time()
    predictions = model.predict(test_features)
    evaluation_time = time.time() - start_time
    accuracy = accuracy_score(test_labels, predictions) * 100
    precision = precision_score(test_labels, predictions, average='binary') * 100
    avg_precision = precision_score(test_labels, predictions, average='macro') * 100
    print('Model Performance')
    print(f'Accuracy = {accuracy:.2f}%')
    print(f'Precision = {precision:.2f}%')
    print(f'Average Precision = {avg_precision:.2f}%')
    print(f'Evaluation time: {evaluation_time:.2f} seconds')
    return accuracy, precision, avg_precision, evaluation_time

# Evaluate best SVM model
best_svm_model = svm_random.best_estimator_
accuracy, precision, avg_precision, evaluation_time = evaluate_svm(best_svm_model, X_test, y_test)

# Calculating average accuracy from cross-validation results
average_accuracy = np.mean(svm_random.cv_results_['mean_test_score']) * 100

print(f"Training time: {training_time:.2f} seconds")
print(f"Highest accuracy obtained: {accuracy:.2f}%")
print(f"Average accuracy during cross-validation: {average_accuracy:.2f}%")
print(f"Precision: {precision:.2f}%")
print(f"Average Precision: {avg_precision:.2f}%")

Model Performance
Accuracy = 97.75%
Precision = 96.77%
Average Precision = 97.64%
Evaluation time: 0.00 seconds
Training time: 78.17 seconds
Highest accuracy obtained: 97.75%
Average accuracy during cross-validation: 78.82%
Precision: 96.77%
Average Precision: 97.64%
